In [1]:
# Import libraries
import numpy as np
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD
from PIL import Image

2023-11-22 14:09:18.115265: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Data preparation

In [2]:
# set directory for train and test data
train_ds = './data/train/'
test_ds = './data/test/'

In [3]:
# Rescalling the images
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches using train_datagen
train_generator = train_datagen.flow_from_directory(
        train_ds,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary',
        shuffle=True)

# Flow test images in batches using train_datagen
test_generator = test_datagen.flow_from_directory(
        test_ds,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary',
        shuffle=True)

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


In [4]:
train_generator.class_indices

{'bee': 0, 'wasp': 1}

In [5]:
test_generator.class_indices

{'bee': 0, 'wasp': 1}

## Building CNN Model

In [6]:
# Add custom layers
model = Sequential([
    Conv2D(32, (3, 3), activation = 'relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

2023-11-22 14:09:19.299674: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


1. **Sequential Model**:
   - The `Sequential` model is a linear stack of layers in Keras. It allows you to create models layer-by-layer in a step-by-step fashion.

2. **Convolutional Layer (Conv2D)**:
   - `Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3))`:
     - This is the first layer of your network and a convolutional layer.
     - It has 32 filters (or kernels), each of size 3x3. Filters are used to extract features from the input image.
     - `activation='relu'` indicates that the Rectified Linear Unit (ReLU) function is used as an activation function. ReLU introduces non-linearity to the model, allowing it to learn more complex patterns.
     - `input_shape=(150, 150, 3)` defines the shape of the input data: 150x150 pixels and 3 channels (assuming color images, typically RGB).

3. **Max Pooling Layer (MaxPooling2D)**:
   - `MaxPooling2D(2, 2)`:
     - This layer reduces the spatial dimensions (width and height) of the input volume.
     - It performs down-sampling by dividing the input into pools of size 2x2 and taking the maximum value of each pool. This helps reduce the number of parameters and computation in the network, and also controls overfitting.

4. **Flattening (Flatten)**:
   - `Flatten()`:
     - This layer flattens the 2D arrays from the previous layers into a 1D vector. This step is necessary because the Dense layers expect 1D inputs.

5. **Fully Connected Layer (Dense)**:
   - `Dense(64, activation='relu')`:
     - This is a fully connected layer with 64 neurons.
     - It takes the flattened input and applies weights, biases, and the ReLU activation function. This layer allows the network to learn non-linear combinations of the high-level features extracted by the convolutional layers.

6. **Output Layer (Dense)**:
   - `Dense(1, activation='sigmoid')`:
     - This is the output layer of the model with a single neuron.
     - Since this is a binary classification problem (bee or wasp), one neuron is sufficient for output. 
     - The `sigmoid` activation function is used, which outputs a value between 0 and 1, representing the probability of one class (e.g., a bee). A value close to 1 indicates a high probability of the class, while a value close to 0 indicates a low probability.

In summary, your model starts with a convolutional layer to extract features from the image, followed by max pooling to reduce dimensionality. Then, it flattens the output and uses dense layers to further process the data. The final output is obtained through a sigmoid activation function in the last dense layer, suitable for binary classification.

In [7]:
# Model compilation
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=0.002, momentum=0.8),
              metrics=['accuracy'])

In [8]:
# Question 2: Number of Parameters in the Convolutional Layer
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
____________________________________________

Q2 Answer: 896

## Trainning the model

In [9]:
history = model.fit(
    train_generator,
   # steps_per_epoch=10,
    epochs=10,
    validation_data=test_generator,
   # validation_steps=10
)

2023-11-22 14:09:19.500727: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/10
184/184 [==============================] - ETA: 0s - loss: 0.6710 - accuracy: 0.5825

2023-11-22 14:09:46.710721: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


184/184 [==============================] - 30s 161ms/step - loss: 0.6710 - accuracy: 0.5825 - val_loss: 0.6295 - val_accuracy: 0.6503
Epoch 2/10
184/184 [==============================] - 31s 167ms/step - loss: 0.6356 - accuracy: 0.6337 - val_loss: 0.6001 - val_accuracy: 0.6852
Epoch 3/10
184/184 [==============================] - 29s 159ms/step - loss: 0.5980 - accuracy: 0.6804 - val_loss: 0.5716 - val_accuracy: 0.7135
Epoch 4/10
184/184 [==============================] - 30s 161ms/step - loss: 0.5525 - accuracy: 0.7212 - val_loss: 0.5362 - val_accuracy: 0.7179
Epoch 5/10
184/184 [==============================] - 28s 153ms/step - loss: 0.5144 - accuracy: 0.7550 - val_loss: 0.5342 - val_accuracy: 0.7527
Epoch 6/10
184/184 [==============================] - 29s 158ms/step - loss: 0.4924 - accuracy: 0.7710 - val_loss: 0.5257 - val_accuracy: 0.7440
Epoch 7/10
184/184 [==============================] - 29s 159ms/step - loss: 0.4581 - accuracy: 0.7949 - val_loss: 0.5325 - val_accuracy: 0.7

In [10]:
# Question 3: What is the median of training accuracy for all the epochs for this model?
accuracies = history.history['accuracy']
median_accuracy = np.median(accuracies).round(3)
print(median_accuracy)

0.763


In [11]:
history.history

{'loss': [0.6710295677185059,
  0.6355607509613037,
  0.598028838634491,
  0.5524823069572449,
  0.5144400000572205,
  0.4923674166202545,
  0.4580841362476349,
  0.4380904734134674,
  0.4155271649360657,
  0.37734392285346985],
 'accuracy': [0.5825400948524475,
  0.6336687803268433,
  0.6804460287094116,
  0.7212401628494263,
  0.7549632787704468,
  0.7710089683532715,
  0.7949415445327759,
  0.8107152581214905,
  0.826216995716095,
  0.8501495718955994],
 'val_loss': [0.6294625401496887,
  0.6001157164573669,
  0.5715534687042236,
  0.5361791253089905,
  0.5342348217964172,
  0.5256584882736206,
  0.5324500799179077,
  0.5129280686378479,
  0.47923290729522705,
  0.49249571561813354],
 'val_accuracy': [0.6503267884254456,
  0.6851851940155029,
  0.7135076522827148,
  0.7178649306297302,
  0.7527233362197876,
  0.7440087199211121,
  0.7440087199211121,
  0.7516340017318726,
  0.7734204530715942,
  0.772331178188324]}

In [12]:
# Question 4: What is the standard deviation of training loss for all the epochs for this model?
losses = history.history['loss']
std_loss = np.std(losses).round(3)
print(std_loss)

0.093


## Data Augmentation

In [13]:
# Data generator with augmentations for the training set
train_datagen_augmented = ImageDataGenerator(
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generator for the test set (without augmentations)
test_datagen = ImageDataGenerator()

# Create augmented training and regular test generators
train_generator_augmented = train_datagen_augmented.flow_from_directory(
    train_ds,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True)

test_generator = test_datagen.flow_from_directory(
    test_ds,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True)

Found 3677 images belonging to 2 classes.
Found 918 images belonging to 2 classes.


In [14]:
# Continue training the model for 10 more epochs
history = model.fit(
    train_generator_augmented,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10


2023-11-22 14:15:23.553640: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


184/184 [==============================] - ETA: 0s - loss: 916.6515 - accuracy: 0.5385

2023-11-22 14:16:02.881765: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


184/184 [==============================] - 42s 228ms/step - loss: 916.6515 - accuracy: 0.5385 - val_loss: 0.6883 - val_accuracy: 0.5392
Epoch 2/10
140/184 [=====================>........] - ETA: 9s - loss: 0.6970 - accuracy: 0.5370

KeyboardInterrupt: 

In [ ]:
# Question 5: What is the mean of test loss for all the epochs for the model trained with augmentations?
test_losses = history.history['val_loss']
mean_test_loss = np.mean(accuracies).round(3)
print(mean_test_loss)

0.704


In [ ]:
# Question 6: What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?
test_accuracies = history.history['val_accuracy']
last_5_accuracies = test_accuracies[5:10]
mean_last_5_acc = np.mean(last_5_accuracies).round(3)
print(mean_last_5_acc)

0.537
